<a href="https://colab.research.google.com/github/sahupra1357/HousingRateRegression/blob/main/cnn_regressionGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/emanhamed/Houses-dataset.git

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
!pip install requests

In [ ]:
import requests
# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/sahupra1357/HousingRateRegression/main/pyimagesearch/datasets.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('datasets.py', 'w') as f:
    f.write(r.text)

url = 'https://raw.githubusercontent.com/sahupra1357/HousingRateRegression/main/pyimagesearch/models.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('models.py', 'w') as f:
    f.write(r.text)

import datasets
import models

In [ ]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os

In [ ]:
# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
inputPath = os.path.sep.join(["Houses-dataset/Houses Dataset", "HousesInfo.txt"])
df = datasets.load_house_attributes(inputPath)


[INFO] loading house attributes...


In [ ]:
# load the house images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading house images...")
images = datasets.load_house_images(df, "Houses-dataset/Houses Dataset")
images = images / 255.0

[INFO] loading house images...


In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [ ]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice
# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*
model = models.create_cnn(64, 64, 3, regress=True)
opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
cnn_hist = model.fit(x=trainImagesX, y=trainY, 
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[INFO] training model...
Epoch 1/200
34/34 [==============================] - 4s 65ms/step - loss: 630.9975 - val_loss: 203.6262
Epoch 2/200
34/34 [==============================] - 2s 54ms/step - loss: 427.9910 - val_loss: 316.8934
Epoch 3/200
34/34 [==============================] - 2s 55ms/step - loss: 272.3477 - val_loss: 115.6811
Epoch 4/200
34/34 [==============================] - 2s 54ms/step - loss: 257.2148 - val_loss: 630.4659
Epoch 5/200
34/34 [==============================] - 2s 54ms/step - loss: 166.0401 - val_loss: 1001.8469
Epoch 6/200
34/34 [==============================] - 2s 56ms/step - loss: 148.4325 - val_loss: 1155.2451
Epoch 7/200
34/34 [==============================] - 2s 55ms/step - loss: 152.7984 - val_loss: 1333.8124
Epoch 8/200
34/34 [==============================] - 2s 54ms/step - loss: 116.7083 - val_loss: 1142.8527
Epoch 9/200
34/34 [==============================] - 2s 56ms/step - loss: 100.7070 - val_loss: 929.9512
Epoch 10/200
34/34 [===============

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,10)) 
epochs = range(1, len(regression_hist.history['accuracy']) + 1)
plt.plot(epochs, regression_hist.history['loss'], label='Training Loss') 
plt.plot(epochs, regression_hist.history['val_loss'], label='Validation Loss') 

plt.title('Loss of Regression model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 60.47%, std: 28.68%
